In [1]:
from PIL import Image, ImageDraw, ImageFont
import os
import piexif
from datetime import datetime

def add_watermark(image_path, text, save_as=None):
    original = Image.open(image_path)
    txt = Image.new('RGBA', original.size, (255,255,255,0))
    draw = ImageDraw.Draw(txt)
    font = ImageFont.truetype("arial.ttf", 20)

    text_x = original.width // 2  
    text_y = original.height - 30  

    text_position = (text_x, text_y)

    draw.text(text_position, text, fill=(255,255,255,128), font=font, anchor="mm")
    watermarked = Image.alpha_composite(original.convert('RGBA'), txt).convert('RGB')
    
    if save_as:
        watermarked.save(save_as, 'JPEG')
    else:
        watermarked.save(image_path, 'JPEG')



def get_camera_info(exif_data):
    camera_info = {
        'make': exif_data['0th'].get(piexif.ImageIFD.Make, b'Unknown').decode('utf-8'),
        'model': exif_data['0th'].get(piexif.ImageIFD.Model, b'Unknown').decode('utf-8'),
        'iso': str(exif_data['Exif'].get(piexif.ExifIFD.ISOSpeedRatings, 'Unknown')),
        'fstop': 'Unknown',
        'shutterspeed': 'Unknown',
        'datetime': exif_data['Exif'].get(piexif.ExifIFD.DateTimeOriginal, b'Unknown').decode('utf-8')
    }


    if piexif.ExifIFD.FNumber in exif_data['Exif']:
        f_number = exif_data['Exif'][piexif.ExifIFD.FNumber]
        fstop_value = f_number[0] / f_number[1]
        camera_info['fstop'] = f"f/{fstop_value}"


    if piexif.ExifIFD.ExposureTime in exif_data['Exif']:
        exposure_time = exif_data['Exif'][piexif.ExifIFD.ExposureTime]
        shutter_speed_value = exposure_time[0] / exposure_time[1]
       
        if shutter_speed_value < 1:
            camera_info['shutterspeed'] = f"1/{round(1/shutter_speed_value)} sec"
        else:
            camera_info['shutterspeed'] = f"{round(shutter_speed_value, 2)} sec"

    if camera_info['datetime'] != 'Unknown':
        year_taken = datetime.strptime(camera_info['datetime'], '%Y:%m:%d %H:%M:%S').year
    else:
        year_taken = 'Unknown'
    
    return camera_info, year_taken


def process_images(source_folder_path):
    # Define the target directory for watermarked images
    target_folder_path = '../images/watermark'
    
    # Ensure the target directory exists
    os.makedirs(target_folder_path, exist_ok=True)
    
    for filename in os.listdir(source_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(source_folder_path, filename)
            exif_data = piexif.load(image_path)
            camera_info, year_taken = get_camera_info(exif_data)

            watermark_text = (f"(c) Charles Lehnen {year_taken}, {camera_info['model']}, "
                              f"ISO {camera_info['iso']}, {camera_info['fstop']}, {camera_info['shutterspeed']}")

            save_as_path = os.path.join(target_folder_path, f"{os.path.splitext(filename)[0]}_watermarked.jpg")
            
            add_watermark(image_path, watermark_text, save_as=save_as_path)

# Adjust the source folder path as needed
process_images('../images/no_watermark')

In [8]:
import os

# Directory where the images are stored
image_folder = '../images/watermark/'
# Prefix for the path to use in the markdown output
output_path_prefix = 'images/watermark/'

images = [f for f in os.listdir(image_folder) if f.lower().endswith('.jpg')]

markdown_output = "::: {layout=\"[[1, 1], [1]]\"}\n"
for image in images:
    # Correct the path for the output
    image_path = os.path.join(output_path_prefix, image).replace("\\", "/")
    # Add an image entry with an empty line after it
    markdown_output += f"![]({image_path}){{group=\"my-gallery\"}}\n\n"
markdown_output = markdown_output.strip() + "\n:::"

print(markdown_output)



::: {layout="[[1, 1], [1]]"}
![](images/watermark/amblyrhynchus_cristatus_watermarked.jpg){group="my-gallery"}

![](images/watermark/california_sea_lion_watermarked.jpg){group="my-gallery"}

![](images/watermark/conolophus_subcristalus_watermarked.jpg){group="my-gallery"}

![](images/watermark/iguana_iguana_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_0110_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_3895_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5437_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5444_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_5868_watermarked.jpg){group="my-gallery"}

![](images/watermark/IMG_7778_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3232_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3254_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3255_watermarked.jpg){group="my-gallery"}

![](images/watermark/_DSC3258_watermarked